In [ ]:
from utils import *
from prepare_ds import *
from train_ml import *
from create_map import *


downgrade_labels = True
force = False


if downgrade_labels:
    src = DEFAULT_PATH["labels"]
    out = DEFAULT_PATH["labels"] + "downgraded/"
    classes_matching = DEFAULT_PATH["labels"] + "classes_matching.csv"
    downgrade_classes(src, out, classes_matching)
    DEFAULT_PATH["labels"] = out

# you can use force=True for re-write all caches.
signs = parse_tifs_from(DEFAULT_PATH["images"], "sign", force=force)
signs = signs.query("type == 'sign'")

labels = parse_tifs_from(DEFAULT_PATH["labels"], "label", force=force)
labels = labels.query("type == 'label'")

Initialization paths...
Created directory: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/cropped/labels/
Created directory: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/cropped/etalons/
All paths was initialized.

Skip downgrade the map of MODIS. If need, set force=True
Load 'sign' from cached DataFrame to path:  /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/path2tif_sign.csv
Load 'label' from cached DataFrame to path:  /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/path2tif_label.csv


In [7]:
force = False

year = 2020
percent = 0.05
only_bands = ["r", "b", "g", "n"]

signs_paths = signs.query(
    f"year == {year} and season == '90d' and band in @only_bands"
).sort_values("band")
signs_paths = signs_paths["path"].to_list()
labels_paths = labels.query(f"year == {year}")["path"].to_list()

z_x, z_y, x, y = generate_dataset(
    signs_paths, labels_paths, force=force, percent=percent
)


Loading & preparing image data...

Cropping & loading labels by 1st image:
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/reg_64_2020_06_90d_comp.b.tif
Files was loaded.
Loading from cache...
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/cropped_labels/landcover23c_v571_2020.Samara.tif
Files was loaded.
Crop labels is done.

Resizing images by 1st label:
Loading from cache...
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/resized/images/reg_64_2020_06_90d_comp.b.tif
Files was loaded.
Loading from cache...
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/resized/images/reg_64_2020_06_90d_comp.g.tif
Files was loaded.
Loading from cache...
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/resized/images/reg_64_2020_06_90d_comp.n.tif
Files was loaded.
Loading from cache...
Loading file: /Users/stephe

In [10]:
m = train_model(z_x, z_y)
m

Split X, y -> X_train, y_train...
Start training model...
Selected model / method: RF
Model was trained. Start validate it...

Classification Report:
              precision    recall  f1-score   support

           1       0.66      0.66      0.66      1511
           2       0.65      0.77      0.70      5224
           3       0.46      0.38      0.42      3252
           4       0.85      0.69      0.76       454
           5       0.39      0.12      0.19        89
           6       0.08      0.00      0.01       241

    accuracy                           0.61     10771
   macro avg       0.51      0.44      0.46     10771
weighted avg       0.59      0.61      0.59     10771

Confusion Matrix:
[[ 993  256  249   11    2    0]
 [ 151 4006 1059    6    2    0]
 [ 284 1696 1236   21    8    7]
 [  47   41   47  313    2    4]
 [   5   27   43    2   11    1]
 [  15  143   63   16    3    1]]
Saving model...
Model was saved: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/

RandomForestClassifier(random_state=42)

In [ ]:
# Mapping by RAM tensor-images (230m)
out = f"handle_map_{percent}.tif"
create_map(x, m, out)

In [ ]:
# Mapping by loaded images from path (10m)

only_bands = ["r", "b", "g", "n"]

tiles = DEFAULT_PATH["images"] + "tiles/"
tiles = parse_tifs_from(tiles, typeof="tile")
tiles = tiles.query("band in @only_bands").sort_values("band")["path"]
tiles = tiles.to_list()

loaded_tiles = []
for t in tiles:
    loaded_tiles.append(load_tif(t, only_first=True))

Load 'tile' from cached DataFrame to path:  /Users/stephenhawking/Coding/ML/low2high_map/src/../data/processing/path2tif_tile.csv
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/tiles/comp_s2_90d_b2_l2a_med.frag.0.tif
Files was loaded.
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/tiles/comp_s2_90d_b3_l2a_med.frag.0.tif
Files was loaded.
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/tiles/comp_s2_90d_b8_l2a_med.frag.0.tif
Files was loaded.
Loading file: /Users/stephenhawking/Coding/ML/low2high_map/src/../data/input/images_10m/Sentinel_Samara/tiles/comp_s2_90d_b4_l2a_med.frag.0.tif
Files was loaded.


In [ ]:
out = f"handle_map_10m_{percent}.tif"
predicted = create_map(loaded_tiles, m, out)

Reshaping tensor-images...
Start create of map...

map size: x = 10980, y = 10980, bands = 4, total px=120560400


Creating map...: 100%|██████████| 58/58 [32:56<00:00, 34.08s/it]


Map is done.
Data saved to /Users/stephenhawking/Coding/ML/low2high_map/src/../data/output/handle_map_10m_0.05.tif


In [ ]:
# Module: validation
from validation import *

year = 2020
map = 'wc'

predicted = "/Users/stephenhawking/Coding/ML/low2high_map/src/../data/output/handle_map_10m_0.05.tif"
etalons = DEFAULT_PATH["etalons"]
etalons = parse_tifs_from(etalons, typeof='etalon')
etalons = etalons.query(f"year == {year} and map =='{map}'")["path"].to_list()
r = validate_how_tif(predicted, etalons, year=2020, map='wc')
create_diff_map(predicted, etalons[0], 'positive')